In [ ]:
# Form implementation generated from reading ui file 'main.ui'
#
# Created by: PyQt6 UI code generator 6.7.1
#
# WARNING: Any manual changes made to this file will be lost when pyuic6 is
# run again.  Do not edit this file unless you know what you are doing.
import os
import sys
import time
import schedule
from PyQt6 import QtCore, QtGui, QtWidgets
from functools import partial
from pack import packZipFile,packTarFile
from unpack import unpackZipFile,unpackTarFile
from huffman import huffmanCompress,huffmanDecompress
from Lzma import lzmaCompress,lzmaDecompress
from encrypt import encrypt3DES,encryptAES
from decrypt import decrypt3DES,decryptAES
from backup import backupLocal,backupGitHub
from threading import Thread,Event

#GUI界面设计
class Ui_Dialog(object):
    #GUI界面初始化
    def init(self,Dialog):
        #背景设计
        self.Dialog=Dialog
        Dialog.setObjectName("Dialog")
        Dialog.resize(528, 535)
        
        #提示框设计
        self.label = QtWidgets.QLabel(parent=Dialog)
        self.label.setGeometry(QtCore.QRect(190, 30, 151, 20))
        self.label.setObjectName("label")
        
        #选项框架设计
        self.verticalLayoutWidget = QtWidgets.QWidget(parent=Dialog)
        self.verticalLayoutWidget.setGeometry(QtCore.QRect(60, 70, 401, 431))
        self.verticalLayoutWidget.setObjectName("verticalLayoutWidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.verticalLayoutWidget)
        self.verticalLayout.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout.setObjectName("verticalLayout")

        '''#自动备份子进程启动
        self.backup_local_thread=backup_local_thread
        self.backup_online_thread=backup_online_thread'''

        #查询参数初始化，设置页面大小
        self.size=8
        
        #进入主界面
        self.setupUI(Dialog)

    #主界面设计
    def setupUI(self,Dialog):
        
        #清屏
        self.clear()

        #重置根目录位置
        self.path='C:\\'

        #操作选项设计
        self.pushButton = QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton.setObjectName("pushButton")
        self.verticalLayout.addWidget(self.pushButton)
        self.pushButton_2 = QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_2.setObjectName("pushButton_2")
        self.verticalLayout.addWidget(self.pushButton_2)
        self.retranslateUi(Dialog)
        self.pushButton.clicked.connect(self.backupInit) # type: ignore
        self.pushButton_2.clicked.connect(self.restoreInit) # type: ignore
        QtCore.QMetaObject.connectSlotsByName(Dialog)

    def backupInit(self):
        self.tag=0
        self.searchDir()

    def restoreInit(self):
        self.tag=1
        self.searchDir()

    def backupDstInit(self):
        self.tag=2
        self.dst_path=self.path
        self.searchDst()

    def restoreDstInit(self):
        self.tag=3
        self.dst_path=self.path
        self.searchDst()
        
    #清屏操作，删除选项框架中的所有组件
    def clear(self):
        while self.verticalLayout.count():
            child = self.verticalLayout.takeAt(0)
            if child.widget():
                child.widget().deleteLater()

    #进入下级目录
    def Forward(self,file):
        if self.tag<2:
            self.path=f'{self.path}{file}\\'
            self.searchDir()
        else:
            self.dst_path=f'{self.dst_path}{file}\\'
            self.searchDst()

    #回到上级目录
    def Back(self):
        if self.tag<2:
            self.path='\\'.join(self.path[:-1].split('\\')[:-1])+'\\'
            self.searchDir()
        else:
            self.dst_path='\\'.join(self.dst_path[:-1].split('\\')[:-1])+'\\'
            self.searchDst()
            
    def showFiles(self,page):
        self.clear()
        length=min(len(self.files)-page*self.size,self.size)
        _translate = QtCore.QCoreApplication.translate
        
        #选项设计，为当前路径下每个文件创建一个按钮
        for i in range(0,length):
            file_button = QtWidgets.QCheckBox(parent=self.verticalLayoutWidget)
            file_button.setObjectName(f"fileButton{i}")
            file_button.setText(_translate("Dialog", self.files[page*self.size+i]))
        
            # 使用 partial 连接信号
            file_button.clicked.connect(partial(self.targetsChange, self.files[page*self.size+i]))
            self.verticalLayout.addWidget(file_button)
            
            # 动态设置属性
            setattr(self, f'file_button{i}', file_button)
            exec('self.file_button{}.show'.format(i))
            
        if page>0:
            self.last=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.last.setObjectName('last')
            self.last.setText(_translate('Dialog','上一页'))
            self.last.clicked.connect(partial(self.showFiles, page-1))
            self.verticalLayout.addWidget(self.last)

        if page*self.size+self.size<len(self.files):
            self.next=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.next.setObjectName('next')
            self.next.setText(_translate('Dialog','下一页'))
            self.next.clicked.connect(partial(self.showFiles, page+1))
            self.verticalLayout.addWidget(self.next)

        #确定按钮设计，表示已完成目标文件的选择
        self.confirm=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.confirm.setObjectName('confirm')
        self.confirm.setText(_translate('Dialog','确定'))
        if self.tag==0:
            self.confirm.clicked.connect(self.backupDstInit)
        elif self.tag==1:
            self.confirm.clicked.connect(self.restoreDstInit)
        self.verticalLayout.addWidget(self.confirm)
    
    def showDirs(self,page):
        self.clear()
        length=min(len(self.files)-page*self.size,self.size)
        _translate = QtCore.QCoreApplication.translate
        #选项设计，为当前路径下每个目录创建一个按钮
        for i in range(0,length):
            file_button = QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            file_button.setObjectName(f"fileButton{i}")
            file_button.setText(_translate("Dialog", self.files[page*self.size+i]))
            
            # 使用 partial 连接信号
            file_button.clicked.connect(partial(self.Forward, self.files[page*self.size+i]))
            self.verticalLayout.addWidget(file_button)
            
            # 动态设置属性
            setattr(self, f'file_button{i}', file_button)
            exec('self.file_button{}.show'.format(i))
            
        if page>0:
            self.last=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.last.setObjectName('last')
            self.last.setText(_translate('Dialog','上一页'))
            self.last.clicked.connect(partial(self.showDirs, page-1))
            self.verticalLayout.addWidget(self.last)

        if page*self.size+self.size<len(self.files):
            self.next=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.next.setObjectName('next')
            self.next.setText(_translate('Dialog','下一页'))
            self.next.clicked.connect(partial(self.showDirs, page+1))
            self.verticalLayout.addWidget(self.next)
            
        #选项设计，回到上级目录
        if os.path.abspath(self.path)!='C:\\':
            self.back=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.back.setObjectName('back')
            self.back.setText(_translate('Dialog','返回上级目录'))
            self.back.clicked.connect(self.Back)
            self.verticalLayout.addWidget(self.back)

        #确定按钮设计，表示已到达目标路径
        self.confirm=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.confirm.setObjectName('confirm')
        self.confirm.setText(_translate('Dialog','确定'))
        if self.tag==0:
            self.confirm.clicked.connect(self.Pack)
        elif self.tag==1:
            self.confirm.clicked.connect(self.Decrypt)
        self.verticalLayout.addWidget(self.confirm)

    def showDstDirs(self,page):
        self.clear()
        length=min(len(self.files)-page*self.size,self.size)
        _translate = QtCore.QCoreApplication.translate
        #选项设计，为当前路径下每个目录创建一个按钮
        for i in range(0,length):
            file_button = QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            file_button.setObjectName(f"fileButton{i}")
            file_button.setText(_translate("Dialog", self.files[page*self.size+i]))
            
            # 使用 partial 连接信号
            file_button.clicked.connect(partial(self.Forward, self.files[page*self.size+i]))
            self.verticalLayout.addWidget(file_button)
            
            # 动态设置属性
            setattr(self, f'file_button{i}', file_button)
            exec('self.file_button{}.show'.format(i))

        if page>0:
            self.last=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.last.setObjectName('last')
            self.last.setText(_translate('Dialog','上一页'))
            self.last.clicked.connect(partial(self.showDstDirs, page-1))
            self.verticalLayout.addWidget(self.last)

        if page*self.size+self.size<len(self.files):
            self.next=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.next.setObjectName('next')
            self.next.setText(_translate('Dialog','下一页'))
            self.next.clicked.connect(partial(self.showDstDirs, page+1))
            self.verticalLayout.addWidget(self.next)
            
        #选项设计，回到上级目录
        if os.path.abspath(self.path)!='C:\\':
            self.back=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.back.setObjectName('back')
            self.back.setText(_translate('Dialog','返回上级目录'))
            self.back.clicked.connect(self.Back)
            self.verticalLayout.addWidget(self.back)

        #确定按钮设计，表示已到达目标路径
        self.confirm=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.confirm.setObjectName('confirm')
        self.confirm.setText(_translate('Dialog','确定'))
        if self.tag==2:
            self.confirm.clicked.connect(self.PackFile)
        elif self.tag==3:
            self.confirm.clicked.connect(self.DecryptFile)
        self.verticalLayout.addWidget(self.confirm)
    
    #目标路径查询，移动到目标文件所在路径
    def searchDir(self):
        self.files=[file for file in os.listdir(self.path) if os.path.isdir(f'{self.path}{file}')]
        _translate = QtCore.QCoreApplication.translate
        
        #提示信息更新
        self.label.setText(_translate("Dialog", "请移动到目标文件所在目录"))
        self.showDirs(0)

    def searchDst(self):
        self.files=[file for file in os.listdir(self.dst_path) if os.path.isdir(f'{self.dst_path}{file}')]
        _translate = QtCore.QCoreApplication.translate

        #提示信息更新
        self.label.setText(_translate("Dialog", "请移动到终点目录"))
        self.showDstDirs(0)

    #zip打包，将若干个源文件打包成一个zip压缩包
    def PackZipFile(self):
        self.file=self.targets[0]+'.zip'
        try:
            packZipFile(self.path,self.targets,self.dst_path,self.file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败','操作失败',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件打包至{self.dst_path}{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.Compress()

    '''
    #rar打包，将所选文件打包成一个rar压缩包，后因rarfile库不支持操作而放弃
    def CompressRarFile(self):
        #compressZipFile(self.targets,self.targets[0]+'.rar')
        try:
            compressRarFile(self.targets,self.targets[0]+'.rar')
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败','操作失败',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功','已将所选文件压缩至'+self.targets[0]+'.rar',QtWidgets.QMessageBox.StandardButton.Ok)
        self.clear()
        self.setupUI(self.Dialog)
    '''

    #tgz打包，将若干个源文件打包成一个tgz压缩包
    def PackTarFile(self):
        self.file=self.targets[0]+'.tgz'
        try:
            packTarFile(self.path,self.targets,self.dst_path,self.file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败','操作失败',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件打包至{self.dst_path}{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.Compress()

    #打包操作界面，选择打包方式
    def PackFile(self):
        self.clear()
        if not self.targets:
            QtWidgets.QMessageBox.critical(None,'操作失败','请选择至少一个文件进行备份',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            _translate = QtCore.QCoreApplication.translate

            #提示信息更新
            self.label.setText(_translate("Dialog", "请选择打包方式："))

            #操作选项设计
            self.pushButton= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.pushButton.setObjectName('pushButton')
            self.pushButton.setText(_translate("Dialog", 'zip打包'))
            self.pushButton.clicked.connect(self.PackZipFile)
            self.verticalLayout.addWidget(self.pushButton)
            self.pushButton_2= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
            self.pushButton_2.setObjectName('pushButton_2')
            self.pushButton_2.setText(_translate("Dialog", 'tgz打包'))
            self.pushButton_2.clicked.connect(self.PackTarFile)
            self.verticalLayout.addWidget(self.pushButton_2)

    #zip解包，将一个zip压缩包解包为源文件
    def UnpackZipFile(self,file):
        try:
            unpackZipFile(self.dst_path,file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.path}{file}无法解包',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将{self.path}{file}解包',QtWidgets.QMessageBox.StandardButton.Ok)

    '''
    #rar解包，将一个rar压缩包解包为源文件，后因rarfile库不支持操作而放弃
    def ExtractRarFile(self,file):
        try:
            extractRarFile(file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{file}无法解压',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件解压至{file[:-4]}',QtWidgets.QMessageBox.StandardButton.Ok)
    '''

    #tgz解包，将一个tgz压缩包解包为源文件
    def UnpackTgzFile(self,file):
        try:
            unpackTarFile(self.dst_path,file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.path}{file}无法解包',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将{self.path}{file}解包',QtWidgets.QMessageBox.StandardButton.Ok)

    #解包操作界面，根据文件后缀名识别其打包方式并自动执行相应解包操作
    def Unpack(self):
        self.clear()
        for file in self.targets:
            tag=file[-4:]
            if tag=='.zip':
                self.UnpackZipFile(file)
            elif tag=='.tgz':
                self.UnpackTgzFile(file)
        self.setupUI(self.Dialog)

    #目标文件更改，根据文件勾选状态决定将其添加到或移出目标列表
    def targetsChange(self,file_name):
        if file_name in self.targets:
            self.targets.remove(file_name)
        else:
            self.targets.append(file_name)

    #备份操作界面，选择目标路径下需要备份的文件
    def Pack(self):
        self.clear()
        self.files=[file for file in os.listdir(self.path)]
        self.targets=[]
        _translate = QtCore.QCoreApplication.translate

        #提示信息更新
        self.label.setText(_translate("Dialog", "请选择需要备份的文件："))

        #文件选项设计，为目标路径下每个文件创建一个多选框类型的按钮
        self.showFiles(0)
        '''for i, file in enumerate(files):
            #exec('if file_button{}.exist(): file_button{}.show()'.format(i,i))
            file_button = QtWidgets.QCheckBox(parent=self.verticalLayoutWidget)
            file_button.setObjectName(f"fileButton{i}")
            file_button.setText(_translate("Dialog", file))
    
            # 使用 partial 连接信号
            file_button.clicked.connect(partial(self.targetsChange, file))
            self.verticalLayout.addWidget(file_button)
            
            # 动态设置属性
            setattr(self, f'file_button{i}', file_button)
            exec('self.file_button{}.show'.format(i))

        #确定按钮设计，表示目标文件选择完毕并进入下一阶段
        self.confirm=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.confirm.setObjectName('Confirm')
        self.confirm.setText(_translate('Dialog','确定'))
        self.confirm.clicked.connect(self.backupDstInit)
        self.verticalLayout.addWidget(self.confirm)'''

    #霍夫曼压缩，将目标文件以霍夫曼编码方式进行压缩并添加相应前缀名
    def HuffmanCompress(self):
        try:
            huffmanCompress(self.dst_path,self.file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{self.file}无法压缩',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            self.file=f'huffman_{self.file}'
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件压缩至{self.dst_path}{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.Encrypt()

    #LZMA压缩，将目标文件以LZMA方式进行压缩并添加相应前缀名
    def LZMACompress(self):
        try:
            lzmaCompress(self.dst_path,self.file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{self.file}无法压缩',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            self.file=f'LZMA_{self.file}'
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件压缩至{self.dst_path}{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.Encrypt()

    #压缩操作界面，选择压缩方式
    def Compress(self):
        self.clear()
        _translate = QtCore.QCoreApplication.translate

        #提示信息更新
        self.label.setText(_translate("Dialog", "请选择压缩方式："))

        #操作选项设计
        self.pushButton= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton.setObjectName('pushButton')
        self.pushButton.setText(_translate("Dialog", 'huffman压缩'))
        self.pushButton.clicked.connect(self.HuffmanCompress)
        self.verticalLayout.addWidget(self.pushButton)
        self.pushButton_2= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_2.setObjectName('pushButton_2')
        self.pushButton_2.setText(_translate("Dialog", 'LZMA压缩'))
        self.pushButton_2.clicked.connect(self.LZMACompress)
        self.verticalLayout.addWidget(self.pushButton_2)
        self.pushButton_3= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_3.setObjectName('pushButton_3')
        self.pushButton_3.setText(_translate("Dialog", '不压缩'))
        self.pushButton_3.clicked.connect(self.Encrypt)
        self.verticalLayout.addWidget(self.pushButton_3)

    #霍夫曼解压，将目标文件按霍夫曼编码方式解压，并去除前缀名
    def HuffmanDecompress(self,file):
        try:
            huffmanDecompress(self.dst_path,file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{file}无法解压',QtWidgets.QMessageBox.StandardButton.Ok)
        '''else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件解压至{self.path+file[8:]}',QtWidgets.QMessageBox.StandardButton.Ok)'''

    #LZMA解压，将目标文件按LZMA方式解压，并去除前缀名
    def LZMADecompress(self,file):
        try:
            lzmaDecompress(self.dst_path,file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{file}无法解压',QtWidgets.QMessageBox.StandardButton.Ok)
        '''else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件解压至{self.path+file[5:]}',QtWidgets.QMessageBox.StandardButton.Ok)'''

    #解压操作界面，根据文件前缀名识别其压缩方式并自动执行相应解压操作
    def Decompress(self):
        self.clear()
        new_targets=[]
        for file in self.targets:
            if file[:8]=='huffman_':
                self.HuffmanDecompress(file)
                new_targets.append(file[8:])
            elif file[:5]=='LZMA_':
                self.LZMADecompress(file)
                new_targets.append(file[5:])
            else:
                new_targets.append(file)
        self.targets=new_targets
        self.Unpack()

    #3DES压缩，将目标文件以3DES方式加密，并添加相应前缀名
    def Encrypt3DES(self):
        try:
            encrypt3DES(self.dst_path,self.file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{self.file}无法加密',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件加密至{self.dst_path}3DES_{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.setupUI(self.Dialog)
    
    #AES压缩，将目标文件以AES方式压缩，并添加相应前缀名
    def EncryptAES(self):
        try:
            encryptAES(self.dst_path,self.file)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.dst_path}{self.file}无法加密',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件加密至{self.dst_path}AES_{self.file}',QtWidgets.QMessageBox.StandardButton.Ok)
            self.setupUI(self.Dialog)

    #压缩操作界面，选择压缩方式
    def Encrypt(self):
        self.clear()
        _translate = QtCore.QCoreApplication.translate

        #提示信息更新
        self.label.setText(_translate("Dialog", "请选择加密方式："))

        #操作选项设计
        self.pushButton= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton.setObjectName('pushButton')
        self.pushButton.setText(_translate("Dialog", '3DES加密'))
        self.pushButton.clicked.connect(self.Encrypt3DES)
        self.verticalLayout.addWidget(self.pushButton)
        self.pushButton_2= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_2.setObjectName('pushButton_2')
        self.pushButton_2.setText(_translate("Dialog", 'AES加密'))
        self.pushButton_2.clicked.connect(self.EncryptAES)
        self.verticalLayout.addWidget(self.pushButton_2)
        self.pushButton_3= QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.pushButton_3.setObjectName('pushButton_3')
        self.pushButton_3.setText(_translate("Dialog", '不加密'))
        self.pushButton_3.clicked.connect(self.setupUI)
        self.verticalLayout.addWidget(self.pushButton_3)

    #3DES解密，将目标文件按3DES方式解密，并去除前缀名
    def Decrypt3DES(self,file):
        try:
            decrypt3DES(self.path,file,self.dst_path)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.path}{file}无法解密',QtWidgets.QMessageBox.StandardButton.Ok)
        '''else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件解密至{self.path+file[5:]}',QtWidgets.QMessageBox.StandardButton.Ok)'''

    #AES解密，将目标文件按AES方式解密，并去除前缀名
    def DecryptAES(self,file):
        try:
            decryptAES(self.path,file,self.dst_path)
        except:
            QtWidgets.QMessageBox.critical(None,'操作失败',f'{self.path}{file}无法解密',QtWidgets.QMessageBox.StandardButton.Ok)
        '''else:
            QtWidgets.QMessageBox.information(None,'操作成功',f'已将所选文件解密至{self.path+file[4:]}',QtWidgets.QMessageBox.StandardButton.Ok)'''

    #解密操作界面，根据文件前缀名识别其加密方式并自动执行相应解密操作
    def DecryptFile(self):
        self.clear()
        if not self.targets:
            QtWidgets.QMessageBox.critical(None,'操作失败','请选择至少一个文件进行还原',QtWidgets.QMessageBox.StandardButton.Ok)
        else:
            new_targets=[]
            for file in self.targets:
                if file[:5]=='3DES_':
                    self.Decrypt3DES(file)
                    new_targets.append(file[5:])
                elif file[:4]=='AES_':
                    self.DecryptAES(file)
                    new_targets.append(file[4:])
                else:
                    new_targets.append(file)
            self.targets=new_targets
            self.Decompress()

    #还原操作界面，选择目标路径下需要还原的文件，文件须为zip或tgz类型
    def Decrypt(self):
        self.clear()
        self.files=[file for file in os.listdir(self.path) if file[-4:]=='.zip' or file[-4:]=='.tgz']
        self.targets=[]
        _translate = QtCore.QCoreApplication.translate

        #提示信息更新
        self.label.setText(_translate("Dialog", "请选择需要还原的文件："))

        #文件选项设计，为目标路径下每个类型为zip或tgz的文件创建一个多选框类型的按钮
        self.showFiles(0)
        '''for i, file in enumerate(files):
            file_button = QtWidgets.QCheckBox(parent=self.verticalLayoutWidget)
            file_button.setObjectName(f"fileButton{i}")
            file_button.setText(_translate("Dialog", file))
            # 使用 partial 连接信号
            file_button.clicked.connect(partial(self.targetsChange, file))
            self.verticalLayout.addWidget(file_button)
            # 动态设置属性
            setattr(self, f'file_button{i}', file_button)
            exec('self.file_button{}.show'.format(i))

        #确定按钮设计，表示目标文件选择完毕并进入下一阶段
        self.confirm=QtWidgets.QPushButton(parent=self.verticalLayoutWidget)
        self.confirm.setObjectName('Confirm')
        self.confirm.setText(_translate('Dialog','确定'))
        self.confirm.clicked.connect(self.restoreDstInit)
        self.verticalLayout.addWidget(self.confirm)'''

    #主目录组件再命名，
    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("Dialog", "智能文件管理系统"))
        self.label.setText(_translate("Dialog", "欢迎使用智能文件管理系统"))
        self.pushButton.setText(_translate("Dialog", "文件备份"))
        self.pushButton_2.setText(_translate("Dialog", "文件还原"))

class backupLocalThread(Thread):
    def __init__(self,interval):
        super().__init__()
        self.interval=interval
        self.event=Event()
        self.daemon=True

    def run(self):
        while not self.event.is_set():
            backupLocal()
            self.event.wait(self.interval)

    def stop(self):
        self.event.set()

class backupOnlineThread(Thread):
    def __init__(self,interval):
        super().__init__()
        self.interval=interval
        self.event=Event()
        self.daemon=True

    def run(self):
        while not self.event.is_set():
            backupGitHub()
            self.event.wait(self.interval)

    def stop(self):
        self.event.set()

class MyMainForm(QtWidgets.QMainWindow,Ui_Dialog):
    def __init__(self):
        super().__init__() #parent=None
        self.init(self)
        self.initBackupThreads()

    def initBackupThreads(self):
        #QtCore.QTimer.singleShot(5,self.start_threads)
        self.backup_local_thread=backupLocalThread(10000)
        self.backup_online_thread=backupOnlineThread(10000)
        self.backup_local_thread.start()
        self.backup_online_thread.start()
        
    
if __name__=='__main__':
    app=QtWidgets.QApplication(sys.argv)
    window=MyMainForm()
    window.show()
    sys.exit(app.exec())
    